# Text Classification Example

- This example shows text Classification as an example of supervised machine learning task.
- The main steps are:
        - Explore dataset,
        - Data Preparation,
        - Feature Engineering,
        - Model Training,
        - Performance Assessment

#### Problem Definition
- As discussed above, the problem in context is supervised learning problem. 
- The inherent requirement for supervised learning is the use of labeled dataset 
- Labeled data is necessary for the ML algorithms to learn the patterns in the data. 
- We are using a sample data from amazon reviews.

![Text Classification](https://raoumer.github.io/images/MLmodel.png)

In [15]:
# Import necessary libraries for dataset preparation, feature engineering, model training 
    
from sklearn import model_selection, preprocessing, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import pandas, numpy #textblob, string

 ## References:
 - https://raoumer.github.io/blog_posts/building_ML_model.html
 - https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/
 - https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/
 - https://towardsdatascience.com/text-classification-in-python-dd95d264c802
 - https://towardsdatascience.com/getting-data-ready-for-modelling-feature-engineering-feature-selection-dimension-reduction-39dfa267b95a
 - https://stackoverflow.com/questions/39130600/what-is-the-difference-between-feature-engineering-and-feature-extraction
   